In [29]:
import os
import numpy as np
import pandas as pd
from ast import literal_eval

In [32]:
d = pd.read_excel("../data/Variable_Definition.xlsx")
d.head()

,Var_name,Name,Class,Code,Unit,Attribute,Sector,Commodity,Process,Category1,Category2,Category3,Category4,Com_Sector,Com_Type1,Com_Type2,Com_Type3
0,FinalEnergy_Residential_SpaceHeating_Oil,Final Energy|Residential|Space Heating|Oil,single_var,R-1-1,PJ/yr,VAR_FIn,Residential,RSDOIL,NaN,Heating,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FinalEnergy_Residential_SpaceHeating_Gas,Final Energy|Residential|Space Heating|Gas,single_var,R-1-2,PJ/yr,VAR_FIn,Residential,RSDGAS,NaN,Heating,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FinalEnergy_Residential_SpaceHeating_Electricity,Final Energy|Residential|Space Heating|Electri...,single_var,R-1-3,PJ/yr,VAR_FIn,Residential,RSDELC,NaN,Heating,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FinalEnergy_Residential_SpaceHeating,Final Energy|Residential|Space Heating,single_var,"R-1-4, O-2-5",PJ/yr,VAR_FIn,Residential,NaN,NaN,Heating,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FinalEnergy_Residential_WaterHeating,Final Energy|Residential|Water Heating,single_var,"R-2-5, O-2-6",PJ/yr,VAR_FIn,Residential,NaN,NaN,Warmwater,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import os, sys
sys.path.append("..")

from src.utils import vd_reader
from src.framework.powerframe import PowerFrame
from src.utils.filters import filter_any, filter_from_csv


inp_file = "../data/8GT_Bal.VD"
masterlist = "../data/Masterlist.xlsx"

data = vd_reader(path = inp_file, masterlist_path=masterlist)

Found masterlist at ../data/Masterlist.xlsx


In [5]:
# A class is created, which contains both the data, the summarized data, the name and other information
# Each variable is then a characteristic of this class
file_name = "../data/8GT_Bal.VD"
scenario_name = os.path.split(file_name)[-1].split(".")[0]
region_list = ["DER", "BW"]



class single_var(object):
    # Create a list in which all attributes of the class are stored, so a loop can be run over it later on
    _registry = []
    # The init function is always called. Here the assignments of the variables are defined within the class.
    # The variables must be given always in the order as in the next line, "self" can be ignored thereby.
    def __init__(self, template, var_name, unit, df_rawdata):
        self._registry.append(self)
        self.var_name = var_name
        # df rawdata wird mit oben stehender Filter Methode erstellt und beeinhaltet alle einzelnen Zeilen aus der VD Datei
        self.df_rawdata = df_rawdata
        # Wenn template "True" ist, dann wird die Variable im Template ausgegeben, sonst auf "False" setzen
        self.template = template

        # Übernimmt die Einheiten aus der Masterliste
        self.unit = unit
        # yearlist wird später über GUI ausgewählt
        yearlist = ['2010', '2015', '2020', '2025', '2030', '2035','2040', '2045', '2050']
        # Bereitet die spätere Struktur vor
        dict_df = {"Model" : "TIMES","Scenario" : scenario_name,"Region" : "Placeholder", "Variable": self.var_name, "Unit": self.unit,
        '2010' :"0", '2015' :"0", '2020':"3", '2025':"0", '2030':"0", '2035':"0",'2040':"0", '2045':"0", '2050':"0"}
        df_temp = pd.DataFrame(data=dict_df, index = [0])

        df_region_sum = []
        # Creates for each region and year the sum of all values and thus the total value of the variable
        for region in region_list:
            dict_df = {
                "Model" : "TIMES PanEU v1.0","Scenario" : scenario_name, "Region" : region, "Variable": self.var_name, 
                "Unit": self.unit,
                '2010' :"0", '2015' :"0", '2020':"3", '2025':"0", '2030':"0", '2035':"0",'2040':"0", '2045':"0", '2050':"0"}
            df_temp_region = pd.DataFrame(data=dict_df, index = [0])
            mask_region = df_rawdata.Region == region
            df_region = df_rawdata.loc[mask_region]

            for year in yearlist:
                mask_year = df_region.Period == year
                df_year = df_region.loc[mask_year]
                year_sum = df_year["PV"].sum()

                df_temp_region[year] = year_sum
                
            df_region_sum.append(df_temp_region)
            
        # Die oben erstellte und befüllte Tabelle wird immer als Variable.results gespeichert 
        results = pd.concat(df_region_sum)
        self.results = results


    def add_Germany(self):
        # Diese Funktion ist notwendig, um die beiden deutschen Regionen DER und BW zu addieren. Dabei bleiben die Regionen vorerst enthalten
            # Einfache Möglichkeit, um 2 versch. Filter miteinander zu kombinieren
        mask1 = self.results.Region == "DER"
        mask2 = self.results.Region == "BW"
        df_temp = self.results.loc[mask1 | mask2]


        #df_temp = self.results.loc[self.results["Region"] == ["DER", "BW"]]
        dict_df = {"Model" : "TIMES PanEU v1.0","Scenario" : scenario_name,"Region" : "DEU", "Variable": self.var_name, "Unit": self.unit,
        '2010' : df_temp["2010"].sum(axis=0), '2015' :"0", '2020':"3", '2025':"0", '2030':"0", '2035':"0",'2040':"0", '2045':"0", '2050':"0"}
        yearlist = ['2010', '2015', '2020', '2025', '2030', '2035','2040', '2045', '2050']

        df_temp_DEU = pd.DataFrame(dict_df, index = [0])

        neu = pd.concat([self.results, df_temp_DEU])

        for year in yearlist:
                df_year = df_temp[year]
                year_sum = df_year.sum()
                df_temp_DEU[year] = year_sum
        # Verbinde die bisherigen Results mit dem DF für Deutschland
        results_DEU = pd.concat([self.results, df_temp_DEU])
        self.results = results_DEU

    
    def change_unit(self, target_unit, operation, number):
        self.target_unit = target_unit
        self.operation = operation
        self.number = number
        df_temp = self.results
        print(df_temp)
        yearlist = ['2010', '2015', '2020', '2025', '2030', '2035','2040', '2045', '2050']

        if operation == "multiplication":
            for year in yearlist:
                df_temp[year] = df_temp[year].apply(lambda x: x*self.number)
        
        if operation == "division":
            for year in yearlist:
                df_temp[year] = df_temp[year].apply(lambda x: x/self.number)

        df_temp["Unit"]=self.target_unit
        print(df_temp)
        self.results = df_temp

In [48]:
data.columns

Index(['Attribute', 'Commodity', 'Process', 'Period', 'Region', 'Vintage',
       'TimeSlice', 'UserConstraint', 'PV', 'Process_Description', 'MIMO',
       'Sector', 'Category1', 'Category2', 'Category3', 'Category4',
       'TechCapUnit', 'TechActUnit', 'Commodity_Description', 'Com_Sector',
       'Com_Type1', 'Com_Type2', 'Com_Type3', 'Com_Unit'],
      dtype='object')

In [56]:
## code to create the configuration file. 

columns = d.columns  # Read columns and create config file
config = {}
for i in range(len(d)):  # For all the values in dataframe, read non-empty vales and split.
    p = d.iloc[i]
    config[p[columns[0]]] = {}
    if "name" not in columns or "Name" not in columns:  # Create name if name of column name is not given
        config[p[columns[0]]]["Name"] = "|".join(str(p[columns[0]]).split("_")) 
    
    for col in columns[1:]:
        if (col=="name" or col=="Name") and config[p[columns[0]]].get("Name"):
            if pd.isnull(p[col]):
                config[p[columns[0]]]["Name"] = "|".join(str(p[columns[0]]).split("_"))
            else:
                config[p[columns[0]]]["Name"]=str(p[col])
            continue
        if not pd.isnull(p[col]):
            config[p[columns[0]]][col] = str(p[col]).split(",")

In [57]:
config

{'FinalEnergy_Residential_SpaceHeating_Oil': {'Name': 'Final Energy|Residential|Space Heating|Oil',
  'Class': ['single_var'],
  'Code': ['R-1-1'],
  'Unit': ['PJ/yr'],
  'Attribute': ['VAR_FIn'],
  'Sector': ['Residential'],
  'Commodity': ['RSDOIL'],
  'Category1': ['Heating']},
 'FinalEnergy_Residential_SpaceHeating_Gas': {'Name': 'Final Energy|Residential|Space Heating|Gas',
  'Class': ['single_var'],
  'Code': ['R-1-2'],
  'Unit': ['PJ/yr'],
  'Attribute': ['VAR_FIn'],
  'Sector': ['Residential'],
  'Commodity': ['RSDGAS'],
  'Category1': ['Heating']},
 'FinalEnergy_Residential_SpaceHeating_Electricity': {'Name': 'Final Energy|Residential|Space Heating|Electricity',
  'Class': ['single_var'],
  'Code': ['R-1-3'],
  'Unit': ['PJ/yr'],
  'Attribute': ['VAR_FIn'],
  'Sector': ['Residential'],
  'Commodity': ['RSDELC'],
  'Category1': ['Heating']},
 'FinalEnergy_Residential_SpaceHeating': {'Name': 'Final Energy|Residential|Space Heating',
  'Class': ['single_var'],
  'Code': ['R-1-4',

In [28]:
single_var(
    template = False,
    var_name="Final Energy|Residential|Space Heating|Coal",
    unit ="PJ/yr",
    df_rawdata = filter_any(
        data,
        Region = region_list,
        Attribute = ["VAR_FIn"],
        Commodity = ["RSDCOA"],
        Category1 = ["Heating"],
        Sector = ["Residential"]
    )
).results

,Model,Scenario,Region,Variable,Unit,2010,2015,2020,2025,2030,2035,2040,2045,2050
0,TIMES PanEU v1.0,8GT_Bal,DER,Final Energy|Residential|Space Heating|Coal,PJ/yr,56.723174,10.95107,0.0,0.000377,0.0,0.0,0.0,0.0,0.0
0,TIMES PanEU v1.0,8GT_Bal,BW,Final Energy|Residential|Space Heating|Coal,PJ/yr,0.936771,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [58]:
output = {}
for k,v in config.items():
    output[k]={}
    var_name = v.pop("Name")  # The name is generated in the config file.
    unit = v.pop("Unit")[0]  # The unit also has to be scalar as it is not supported by single_var class for now.
    req_class = v.pop("Class")[0]  # This means that the class cannot be multiple
    code = v.pop("Code")  # Code to be used later on for arrangement.
    filtered_data = filter_any( data, **v )
    if req_class == "single_var":
        output[k]["raw"] = filtered_data
        output[k]["output"] = single_var(
            template = False, 
            var_name=var_name,
            unit =unit,
            df_rawdata = filtered_data).results
    else:
        print(f"{req_class} not implemented. Skipping {k}")

In [60]:
output["Emissions_CO2"]["raw"]["Commodity"].unique()

array(['COMCO2N', 'RSDCO2N', 'TRACO2N', 'AGRCO2N'], dtype=object)